### Project starting in Jupyter

blah

## TITLE

## INTRODUCTION

In [9]:
import pandas as pd
import altair as alt

In [10]:
url2 = "https://raw.githubusercontent.com/agallagh/DSCI-Project/refs/heads/main/players.csv"
players_data = pd.read_csv(url2)
players_data

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [11]:
# tidying the data by dropping the unecessary columns

tidy_players = players_data[["experience", "played_hours", "age"]]
tidy_players

# filtering data in age column for our demographic

filtered_age_df = tidy_players[tidy_players["age"] < 60]
filtered_age_df

,experience,played_hours,age
0,Pro,30.3,9
1,Veteran,3.8,17
2,Veteran,0.0,17
3,Amateur,0.7,21
4,Regular,0.1,21
...,...,...,...
190,Amateur,0.0,20
191,Amateur,0.0,17
192,Veteran,0.3,22
193,Amateur,0.0,17


In [12]:
# creating a scatterplot for our variables and colouring by experience
age_chart = alt.Chart(filtered_age_df).mark_point().encode(
    x=alt.X("age").title("Age"),
    y=alt.Y("played_hours").title("Played Hours"),
    color=alt.Color("experience").title("Experience")
).properties(width=700)

age_chart

alt.Chart(...)

In [13]:
# creating a bar plot of experience vs played hours

experience_chart = alt.Chart(filtered_age_df).mark_bar().encode(
    x=alt.X("experience").title("Experience"),
    y=alt.Y("played_hours").title("Played Hours")
).properties(width = 500).configure_axisX(labelAngle = -45)

experience_chart

alt.Chart(...)

This chart varies on an individual level within each experiece based on our first scatterplot. Therefore the few outliers of amateurs and regulars that have played over 150 hours are largely responsible for the difference. To make the data more representative of the overall demographict, we will be filtering out those outliers by making the played hours less than 100.

In [14]:
# filtering the played hours to be less than 100

filtered_hrs_df = filtered_age_df[filtered_age_df["played_hours"] < 100] 

In [15]:
filtered_chart = alt.Chart(filtered_hrs_df).mark_point().encode(
    x=alt.X("age").title("Age"),
    y=alt.Y("played_hours").title("Played Hours"),
    color=alt.Color("experience").title("Experience Level")
).properties(width = 500)



# Facet by experience to make the visualization more clear.
facetted_chart = alt.Chart(filtered_hrs_df).mark_point().encode(
    x=alt.X("age").title("Age"),
    y=alt.Y("played_hours").title("Played Hours"),
    color=alt.Color("experience").title("Experience Level")
).properties(width = 200).facet("experience", columns = 5)

facetted_chart

alt.FacetChart(...)

## METHODS AND RESULTS

## DISCUSSION